<a href="https://colab.research.google.com/github/dxmai/CS116.M12.KHCL/blob/main/CS116_BT3_K_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thông tin sinh viên:

Họ và tên: Đặng Xuân Mai

MSSV: 19521820

Bài tập 3 - Môn học CS116 - Lập trình python cho máy học

#Cài đặt các thư viện cần thiết


In [ ]:
!pip install matplotlib==3.1.3
!pip install osmnet
!pip install folium
!pip install rtree
!pip install pygeos
!pip install geojson
!pip install geopandas

     |████████████████████████████████| 13.1 MB 89 kB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 994 kB 11.5 MB/s 
     |████████████████████████████████| 6.3 MB 14.3 MB/s 
     |████████████████████████████████| 15.4 MB 40 kB/s 
     |████████████████████████████████| 994 kB 8.9 MB/s 
     |████████████████████████████████| 1.8 MB 7.3 MB/s 


#Clone data


In [ ]:
! git clone https://github.com/CityScope/CSL_HCMC

Cloning into 'CSL_HCMC'...
remote: Enumerating objects: 1930, done.
remote: Counting objects: 100% (590/590), done.
remote: Compressing objects: 100% (314/314), done.
remote: Total 1930 (delta 309), reused 500 (delta 270), pack-reused 1340
Receiving objects: 100% (1930/1930), 178.53 MiB | 24.36 MiB/s, done.
Resolving deltas: 100% (986/986), done.
Checking out files: 100% (542/542), done.


#Đọc dữ liệu

In [ ]:
import geopandas
data = geopandas.read_file("CSL_HCMC/Data/GIS/Population/population_HCMC/population_shapefile/Population_District_Level.shp")

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
data

,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,geometry
0,District 1,760,District,180225.0,187435.0,187419.0,190943.0,197421.0,200297.0,193632.0,208475.0,211220.0,142625.0,23360.4000,24294.9000,24292.9000,24749.600,25589.300,25962.100,25098.200,27022.100,27377.900,18486.8000,15274.835115,7.714899e+06,"POLYGON ((684948.318 1193882.223, 684971.256 1..."
1,District 3,770,District,190553.0,188945.0,190631.0,191580.0,193206.0,193694.0,196333.0,198104.0,199095.0,190375.0,38688.6000,38362.1000,38704.4000,38897.100,39227.200,39326.300,39862.100,40221.700,40422.900,38652.5000,12182.058264,4.925215e+06,"POLYGON ((684382.122 1193521.050, 684410.753 1..."
2,District 4,773,District,180980.0,183261.0,184711.0,185649.0,185808.0,187157.0,186727.0,186995.0,183894.0,175329.0,43311.1000,43857.0000,44204.0000,44428.400,44466.500,44789.300,44686.400,44750.600,44008.400,41958.7000,9483.716891,4.178668e+06,"POLYGON ((687625.438 1189132.516, 687615.280 1..."
3,District 6,775,District,249329.0,253474.0,254212.0,257865.0,266121.0,257183.0,258945.0,260742.0,262757.0,233561.0,34910.5000,35490.8000,35594.2000,36105.700,37261.600,36010.200,36256.900,36508.500,36790.600,32702.7000,13077.365262,7.140945e+06,"POLYGON ((679534.413 1187518.498, 679077.904 1..."
4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.3900,7796.0600,7616.0700,7563.590,7963.850,8418.120,8798.840,9006.200,9282.350,10216.5000,33260.915360,3.525220e+07,"POLYGON ((691506.715 1190058.151, 691359.893 1..."
5,District 10,771,District,230345.0,232450.0,236336.0,237086.0,239180.0,238755.0,238558.0,239826.0,241327.0,234819.0,40354.6000,40723.4000,41404.2000,41535.600,41902.500,41828.000,41793.500,42015.600,42278.600,41138.5000,10478.512538,5.707881e+06,"POLYGON ((682507.505 1189929.778, 682485.431 1..."
6,District 11,772,District,226854.0,232536.0,236443.0,233499.0,227860.0,228030.0,230596.0,234291.0,237110.0,209867.0,44207.9000,45315.2000,46076.6000,45502.900,44404.000,44437.100,44937.200,45657.200,46206.600,40897.6000,10833.546511,5.131351e+06,"POLYGON ((681349.265 1190928.179, 681382.887 1..."
7,Phu Nhuan,768,District,174535.0,175175.0,177242.0,179917.0,180456.0,182821.0,182477.0,181780.0,183288.0,163961.0,35692.6000,35823.5000,36246.2000,36793.300,36903.500,37387.100,37316.800,37174.300,37482.600,33530.3000,12392.134315,4.889837e+06,"POLYGON ((682877.988 1195927.612, 682969.684 1..."
8,Tan Binh,766,District,421724.0,430436.0,434298.0,440351.0,443061.0,448989.0,455029.0,469526.0,470699.0,474792.0,18796.9000,19185.2000,19357.3000,19627.100,19747.900,20012.100,20281.400,20927.500,20979.800,21162.2000,23807.894025,2.243513e+07,"POLYGON ((680274.980 1198401.860, 680578.965 1..."
9,Binh Thanh,765,District,457362.0,470054.0,484135.0,484423.0,482833.0,485772.0,487985.0,489657.0,490618.0,499164.0,22005.5000,22616.2000,23293.7000,23307.600,23231.100,23372.500,23478.900,23559.400,23605.600,24016.8000,29544.712130,2.078389e+07,"POLYGON ((684986.154 1194412.964, 684987.226 1..."


#Load dữ liệu click

In [ ]:
click_data = geopandas.read_file('14-09.json')

In [ ]:
click_data

,FID,dbh,geometry
0,0,1,POINT (106.62994 10.72644)
1,1,1,POINT (106.72987 10.79673)
2,2,1,POINT (106.65731 10.84212)
3,3,1,POINT (106.60434 10.74079)
4,4,1,POINT (106.71608 10.78999)
...,...,...,...
27771,27771,1,POINT (106.58536 10.79847)
27772,27772,1,POINT (106.69992 10.75601)
27773,27773,1,POINT (106.64973 10.77202)
27774,27774,1,POINT (106.68181 10.80059)


#Tính tốc độ tăng trưởng dân số

In [ ]:
def Calculate_Growth(row):
  return row['Pop_2019'] / row['Pop_2017']

In [ ]:
data['Growth'] = data.apply(Calculate_Growth, axis = 1)

In [ ]:
data

,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,geometry,Growth
0,District 1,760,District,180225.0,187435.0,187419.0,190943.0,197421.0,200297.0,193632.0,208475.0,211220.0,142625.0,23360.4000,24294.9000,24292.9000,24749.600,25589.300,25962.100,25098.200,27022.100,27377.900,18486.8000,15274.835115,7.714899e+06,"POLYGON ((684948.318 1193882.223, 684971.256 1...",0.675244
1,District 3,770,District,190553.0,188945.0,190631.0,191580.0,193206.0,193694.0,196333.0,198104.0,199095.0,190375.0,38688.6000,38362.1000,38704.4000,38897.100,39227.200,39326.300,39862.100,40221.700,40422.900,38652.5000,12182.058264,4.925215e+06,"POLYGON ((684382.122 1193521.050, 684410.753 1...",0.956202
2,District 4,773,District,180980.0,183261.0,184711.0,185649.0,185808.0,187157.0,186727.0,186995.0,183894.0,175329.0,43311.1000,43857.0000,44204.0000,44428.400,44466.500,44789.300,44686.400,44750.600,44008.400,41958.7000,9483.716891,4.178668e+06,"POLYGON ((687625.438 1189132.516, 687615.280 1...",0.953424
3,District 6,775,District,249329.0,253474.0,254212.0,257865.0,266121.0,257183.0,258945.0,260742.0,262757.0,233561.0,34910.5000,35490.8000,35594.2000,36105.700,37261.600,36010.200,36256.900,36508.500,36790.600,32702.7000,13077.365262,7.140945e+06,"POLYGON ((679534.413 1187518.498, 679077.904 1...",0.888886
4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.3900,7796.0600,7616.0700,7563.590,7963.850,8418.120,8798.840,9006.200,9282.350,10216.5000,33260.915360,3.525220e+07,"POLYGON ((691506.715 1190058.151, 691359.893 1...",1.100641
5,District 10,771,District,230345.0,232450.0,236336.0,237086.0,239180.0,238755.0,238558.0,239826.0,241327.0,234819.0,40354.6000,40723.4000,41404.2000,41535.600,41902.500,41828.000,41793.500,42015.600,42278.600,41138.5000,10478.512538,5.707881e+06,"POLYGON ((682507.505 1189929.778, 682485.431 1...",0.973032
6,District 11,772,District,226854.0,232536.0,236443.0,233499.0,227860.0,228030.0,230596.0,234291.0,237110.0,209867.0,44207.9000,45315.2000,46076.6000,45502.900,44404.000,44437.100,44937.200,45657.200,46206.600,40897.6000,10833.546511,5.131351e+06,"POLYGON ((681349.265 1190928.179, 681382.887 1...",0.885104
7,Phu Nhuan,768,District,174535.0,175175.0,177242.0,179917.0,180456.0,182821.0,182477.0,181780.0,183288.0,163961.0,35692.6000,35823.5000,36246.2000,36793.300,36903.500,37387.100,37316.800,37174.300,37482.600,33530.3000,12392.134315,4.889837e+06,"POLYGON ((682877.988 1195927.612, 682969.684 1...",0.894554
8,Tan Binh,766,District,421724.0,430436.0,434298.0,440351.0,443061.0,448989.0,455029.0,469526.0,470699.0,474792.0,18796.9000,19185.2000,19357.3000,19627.100,19747.900,20012.100,20281.400,20927.500,20979.800,21162.2000,23807.894025,2.243513e+07,"POLYGON ((680274.980 1198401.860, 680578.965 1...",1.008696
9,Binh Thanh,765,District,457362.0,470054.0,484135.0,484423.0,482833.0,485772.0,487985.0,489657.0,490618.0,499164.0,22005.5000,22616.2000,23293.7000,23307.600,23231.100,23372.500,23478.900,23559.400,23605.600,24016.8000,29544.712130,2.078389e+07,"POLYGON ((684986.154 1194412.964, 684987.226 1...",1.017419


#Lấy ra top 10 quận huyện có tốc độ tăng trưởng nhanh nhất

In [ ]:
data_filtered = data.nlargest(10, 'Growth')
data_filtered

,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,geometry,Growth
16,District 9,763,District,256257.0,263486.0,271537.0,276432.0,284473.0,284990.0,290620.0,296198.0,310739.0,397006.0,2242.76,2306.020,2376.490,2419.330,2489.700,2494.230,2543.500,2592.320,2719.580,3474.59,53139.220230,1.142649e+08,"POLYGON ((701125.957 1205414.891, 701162.391 1...",1.277619
23,Hoc Mon,784,District,349065.0,358640.0,366502.0,396763.0,408683.0,414795.0,422471.0,434275.0,446056.0,542243.0,3197.13,3284.830,3356.840,3634.000,3743.180,3799.160,3869.460,3977.580,4085.480,4966.47,65955.793966,1.091744e+08,"POLYGON ((673061.206 1208548.077, 673094.550 1...",1.215639
18,Nha Be,786,District,101074.0,103793.0,110958.0,119416.0,126062.0,132034.0,139225.0,156001.0,177990.0,206837.0,1005.81,1032.870,1104.170,1188.340,1254.480,1313.900,1385.460,1552.410,1771.220,2058.29,67540.905029,1.004901e+08,"POLYGON ((688444.826 1185726.125, 688568.759 1...",1.162071
22,District 12,761,District,405360.0,427083.0,455881.0,475824.0,489511.0,499569.0,510326.0,526064.0,549064.0,620146.0,7680.92,8092.530,8638.210,9016.090,9275.440,9466.020,9669.850,9968.060,10403.900,11750.80,47703.394094,5.277342e+07,"POLYGON ((685183.891 1205265.848, 685198.395 1...",1.129460
17,District 2,769,District,147490.0,140621.0,137749.0,134131.0,140436.0,140288.0,147168.0,153832.0,161957.0,180275.0,2953.46,2815.910,2758.400,2685.950,2812.210,2809.250,2947.020,3080.460,3243.160,3609.98,39592.970451,4.993869e+07,"POLYGON ((690496.729 1195354.084, 690716.888 1...",1.113104
13,Cu Chi,783,District,343155.0,355822.0,365779.0,381796.0,383981.0,390722.0,403038.0,410984.0,418655.0,462047.0,789.36,818.498,841.402,878.246,883.272,898.778,927.109,945.387,963.033,1062.85,119954.769999,4.346845e+08,"POLYGON ((656983.014 1228510.664, 656969.299 1...",1.103646
4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.39,7796.060,7616.070,7563.590,7963.850,8418.120,8798.840,9006.200,9282.350,10216.50,33260.915360,3.525220e+07,"POLYGON ((691506.715 1190058.151, 691359.893 1...",1.100641
15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.550,10028.200,10487.800,10630.200,10842.200,11065.000,11245.900,11362.300,12410.90,52828.501430,4.775581e+07,"POLYGON ((688796.063 1204383.598, 688818.721 1...",1.092283
14,Binh Chanh,785,District,420109.0,447291.0,469517.0,491900.0,514242.0,551545.0,591451.0,623022.0,652900.0,705508.0,1660.96,1768.420,1856.300,1944.790,2033.120,2180.610,2338.380,2463.200,2581.330,2789.32,129933.645113,2.529152e+08,"POLYGON ((679646.388 1186348.769, 679698.864 1...",1.080576
12,Binh Tan,777,District,572132.0,595335.0,616777.0,639088.0,655244.0,672309.0,686474.0,704347.0,729366.0,784173.0,11020.80,11467.700,11880.800,12310.500,12621.800,12950.500,13223.300,13567.600,14049.500,15105.30,40057.584063,5.191097e+07,"POLYGON ((674953.073 1197210.125, 674961.146 1...",1.075143


Ref: https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.nlargest.html

#Dùng spatial join

##Tìm crs của data trước

In [ ]:
data.crs

<Projected CRS: EPSG:32648>
Name: WGS 84 / UTM zone 48N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 102°E and 108°E, northern hemisphere between equator and 84°N, onshore and offshore. Cambodia. China. Indonesia. Laos. Malaysia - West Malaysia. Mongolia. Russian Federation. Singapore. Thailand. Vietnam.
- bounds: (102.0, 0.0, 108.0, 84.0)
Coordinate Operation:
- name: UTM zone 48N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

##Chuyển dữ liệu click về hệ toạ độ theo data 

In [ ]:
click_data_converted = click_data.to_crs(32648)

In [ ]:
click_data_converted

,FID,dbh,geometry
0,0,1,POINT (678245.073 1186203.645)
1,1,1,POINT (689133.252 1194038.335)
2,2,1,POINT (681170.572 1199015.116)
3,3,1,POINT (675437.515 1187776.551)
4,4,1,POINT (687628.649 1193284.609)
...,...,...,...
27771,27771,1,POINT (673328.517 1194145.495)
27772,27772,1,POINT (685882.869 1189515.920)
27773,27773,1,POINT (680383.184 1191257.068)
27774,27774,1,POINT (683874.156 1194436.524)


Ref: https://geopandas.org/docs/reference/api/geopandas.GeoDataFrame.to_crs.html

##Lọc ra các điểm click trong 10 quận có tốc độ tăng trưởng nhanh nhất

In [ ]:
from geopandas.tools import sjoin

In [ ]:
data_use = sjoin(click_data_converted, data_filtered, how='left', op='within')
data_use

,FID,dbh,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,Growth
0,0,1,POINT (678245.073 1186203.645),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,POINT (689133.252 1194038.335),17.0,District 2,769,District,147490.0,140621.0,137749.0,134131.0,140436.0,140288.0,147168.0,153832.0,161957.0,180275.0,2953.46,2815.91,2758.4,2685.95,2812.21,2809.25,2947.02,3080.46,3243.16,3609.98,39592.970451,4.993869e+07,1.113104
2,2,1,POINT (681170.572 1199015.116),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1,POINT (675437.515 1187776.551),12.0,Binh Tan,777,District,572132.0,595335.0,616777.0,639088.0,655244.0,672309.0,686474.0,704347.0,729366.0,784173.0,11020.80,11467.70,11880.8,12310.50,12621.80,12950.50,13223.30,13567.60,14049.50,15105.30,40057.584063,5.191097e+07,1.075143
4,4,1,POINT (687628.649 1193284.609),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27771,27771,1,POINT (673328.517 1194145.495),12.0,Binh Tan,777,District,572132.0,595335.0,616777.0,639088.0,655244.0,672309.0,686474.0,704347.0,729366.0,784173.0,11020.80,11467.70,11880.8,12310.50,12621.80,12950.50,13223.30,13567.60,14049.50,15105.30,40057.584063,5.191097e+07,1.075143
27772,27772,1,POINT (685882.869 1189515.920),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27773,27773,1,POINT (680383.184 1191257.068),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27774,27774,1,POINT (683874.156 1194436.524),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Ref: https://geopandas.org/gallery/spatial_joins.html

##Loại bỏ những click không nằm trong top 10 quận huyện có tốc độ tăng trưởng nhanh nhất

In [ ]:
data_use = data_use.dropna()
data_use

,FID,dbh,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,Growth
1,1,1,POINT (689133.252 1194038.335),17.0,District 2,769,District,147490.0,140621.0,137749.0,134131.0,140436.0,140288.0,147168.0,153832.0,161957.0,180275.0,2953.46,2815.910,2758.400,2685.950,2812.210,2809.250,2947.020,3080.460,3243.160,3609.98,39592.970451,4.993869e+07,1.113104
3,3,1,POINT (675437.515 1187776.551),12.0,Binh Tan,777,District,572132.0,595335.0,616777.0,639088.0,655244.0,672309.0,686474.0,704347.0,729366.0,784173.0,11020.80,11467.700,11880.800,12310.500,12621.800,12950.500,13223.300,13567.600,14049.500,15105.30,40057.584063,5.191097e+07,1.075143
7,7,1,POINT (689892.210 1201028.944),15.0,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.550,10028.200,10487.800,10630.200,10842.200,11065.000,11245.900,11362.300,12410.90,52828.501430,4.775581e+07,1.092283
9,9,1,POINT (674009.104 1216902.996),13.0,Cu Chi,783,District,343155.0,355822.0,365779.0,381796.0,383981.0,390722.0,403038.0,410984.0,418655.0,462047.0,789.36,818.498,841.402,878.246,883.272,898.778,927.109,945.387,963.033,1062.85,119954.769999,4.346845e+08,1.103646
11,11,1,POINT (677206.921 1205027.209),23.0,Hoc Mon,784,District,349065.0,358640.0,366502.0,396763.0,408683.0,414795.0,422471.0,434275.0,446056.0,542243.0,3197.13,3284.830,3356.840,3634.000,3743.180,3799.160,3869.460,3977.580,4085.480,4966.47,65955.793966,1.091744e+08,1.215639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27763,27763,1,POINT (701154.412 1200316.550),16.0,District 9,763,District,256257.0,263486.0,271537.0,276432.0,284473.0,284990.0,290620.0,296198.0,310739.0,397006.0,2242.76,2306.020,2376.490,2419.330,2489.700,2494.230,2543.500,2592.320,2719.580,3474.59,53139.220230,1.142649e+08,1.277619
27765,27765,1,POINT (680070.265 1185876.243),14.0,Binh Chanh,785,District,420109.0,447291.0,469517.0,491900.0,514242.0,551545.0,591451.0,623022.0,652900.0,705508.0,1660.96,1768.420,1856.300,1944.790,2033.120,2180.610,2338.380,2463.200,2581.330,2789.32,129933.645113,2.529152e+08,1.080576
27766,27766,1,POINT (688706.461 1192472.828),17.0,District 2,769,District,147490.0,140621.0,137749.0,134131.0,140436.0,140288.0,147168.0,153832.0,161957.0,180275.0,2953.46,2815.910,2758.400,2685.950,2812.210,2809.250,2947.020,3080.460,3243.160,3609.98,39592.970451,4.993869e+07,1.113104
27769,27769,1,POINT (687680.453 1191492.308),17.0,District 2,769,District,147490.0,140621.0,137749.0,134131.0,140436.0,140288.0,147168.0,153832.0,161957.0,180275.0,2953.46,2815.910,2758.400,2685.950,2812.210,2809.250,2947.020,3080.460,3243.160,3609.98,39592.970451,4.993869e+07,1.113104


#Dùng K-Mean

##Gom nhóm các click chung 1 quận lại với nhau

In [ ]:
group = data_use.groupby('Dist_Name')
group

Ref: https://www.geeksforgeeks.org/python-pandas-dataframe-groupby/

In [ ]:
#Tên của các group chia theo quận
for name in group.groups:
  print(name)

Binh Chanh
Binh Tan
Cu Chi
District 12
District 2
District 7
District 9
Hoc Mon
Nha Be
Thu Duc


##K-Mean

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=20)

###Gom nhóm theo từng quận

###Hàm đếm từng số lượng của từng cluster

In [ ]:
def Calculate_Each_Cluster(data):
  count_point = {}
  for row in range(len(data)):
    point = data.iloc[row]['cluster']
    if point not in count_point.keys():
      count_point[point] = 1
    else:
      count_point[point] += 1
  print("1 dictionary với key là tên cluster, value là số lượng phần tử trong cluster")
  print(count_point)
  return count_point

###Hàm tìm cluster có nhiều phần tử gần nó nhất

In [ ]:
def Find_Max_Cluster(data):
  count_point = Calculate_Each_Cluster(data)
  max_key = max(count_point, key=count_point.get) #Ref #1
  print("Max key is", max_key, " = ", count_point[max_key])
  return max_key

In [ ]:
#Tạo 1 dataframe rỗng
final_data_use = pd.DataFrame()

In [ ]:
def K_Means(input_data):
  kmeans.fit(input_data)
  y_kmeans = kmeans.predict(input_data)
  return y_kmeans

In [ ]:
for group_name in group.groups:
  #Thực hiện lần lượt trên mỗi quận huyện
  data_in_group = group.get_group(group_name)

  #Chuyển đổi POINT thành toạ độ x, y  (Ref #2)
  lon = pd.Series(data_in_group['geometry'].apply(lambda p: p.x))
  lat = pd.Series(data_in_group['geometry'].apply(lambda p: p.y))
  input_data = np.column_stack((lon, lat))
  
  #Chạy K-Means
  res = K_Means(input_data)
  
  #Thêm cột cluster để tính toán
  data_in_group['cluster'] = res

  #Tìm cluster có nhiều thành phần con nhất 
  max_cluster = Find_Max_Cluster(data_in_group)

  temp = data_in_group[data_in_group['cluster'] == max_cluster]

  #Thêm các click thoả điều kiện vào bộ data cuối cùng để lúc sau sẽ thực hiện vẽ heatmap
  final_data_use = final_data_use.append(temp, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


1 dictionary với key là tên cluster, value là số lượng phần tử trong cluster
{19: 75, 5: 252, 4: 46, 16: 90, 0: 38, 7: 160, 13: 87, 10: 124, 2: 130, 6: 90, 3: 103, 14: 21, 9: 88, 18: 39, 17: 25, 15: 107, 12: 34, 8: 34, 11: 21, 1: 12}
Max key is 5  =  252


/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


1 dictionary với key là tên cluster, value là số lượng phần tử trong cluster
{7: 121, 5: 171, 13: 98, 1: 151, 8: 133, 10: 97, 4: 276, 17: 126, 3: 183, 19: 28, 9: 96, 0: 201, 18: 24, 16: 31, 2: 213, 15: 125, 14: 121, 6: 93, 11: 31, 12: 15}
Max key is 4  =  276


/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


...

1 dictionary với key là tên cluster, value là số lượng phần tử trong cluster
{17: 69, 9: 51, 15: 45, 2: 118, 1: 79, 11: 84, 12: 42, 13: 73, 8: 57, 19: 75, 6: 33, 3: 29, 16: 57, 18: 13, 0: 88, 7: 27, 5: 19, 4: 23, 10: 25, 14: 9}
Max key is 2  =  118


/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


1 dictionary với key là tên cluster, value là số lượng phần tử trong cluster
{18: 18, 3: 12, 10: 17, 2: 68, 5: 25, 4: 13, 19: 16, 8: 35, 14: 8, 6: 9, 17: 8, 12: 19, 1: 18, 0: 9, 7: 26, 11: 14, 15: 6, 9: 15, 13: 10, 16: 21}
Max key is 2  =  68


/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


1 dictionary với key là tên cluster, value là số lượng phần tử trong cluster
{2: 80, 17: 117, 4: 151, 0: 88, 6: 64, 14: 63, 9: 90, 13: 63, 3: 174, 7: 41, 1: 88, 19: 35, 10: 99, 11: 55, 16: 12, 18: 52, 8: 38, 15: 68, 5: 53, 12: 38}
Max key is 3  =  174


In [ ]:
final_data_use

,FID,dbh,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,Growth,cluster
0,36,1,POINT (665721.622 1193183.114),14.0,Binh Chanh,785,District,420109.0,447291.0,469517.0,491900.0,514242.0,551545.0,591451.0,623022.0,652900.0,705508.0,1660.96,1768.42,1856.3,1944.79,2033.12,2180.61,2338.38,2463.2,2581.33,2789.32,129933.645113,2.529152e+08,1.080576,5
1,47,1,POINT (665640.212 1193259.771),14.0,Binh Chanh,785,District,420109.0,447291.0,469517.0,491900.0,514242.0,551545.0,591451.0,623022.0,652900.0,705508.0,1660.96,1768.42,1856.3,1944.79,2033.12,2180.61,2338.38,2463.2,2581.33,2789.32,129933.645113,2.529152e+08,1.080576,5
2,54,1,POINT (665494.740 1193208.527),14.0,Binh Chanh,785,District,420109.0,447291.0,469517.0,491900.0,514242.0,551545.0,591451.0,623022.0,652900.0,705508.0,1660.96,1768.42,1856.3,1944.79,2033.12,2180.61,2338.38,2463.2,2581.33,2789.32,129933.645113,2.529152e+08,1.080576,5
3,294,1,POINT (665353.309 1193252.243),14.0,Binh Chanh,785,District,420109.0,447291.0,469517.0,491900.0,514242.0,551545.0,591451.0,623022.0,652900.0,705508.0,1660.96,1768.42,1856.3,1944.79,2033.12,2180.61,2338.38,2463.2,2581.33,2789.32,129933.645113,2.529152e+08,1.080576,5
4,371,1,POINT (666658.173 1191267.525),14.0,Binh Chanh,785,District,420109.0,447291.0,469517.0,491900.0,514242.0,551545.0,591451.0,623022.0,652900.0,705508.0,1660.96,1768.42,1856.3,1944.79,2033.12,2180.61,2338.38,2463.2,2581.33,2789.32,129933.645113,2.529152e+08,1.080576,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1865,26881,1,POINT (688628.898 1197541.102),15.0,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.2,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.90,52828.501430,4.775581e+07,1.092283,3
1866,27186,1,POINT (688263.843 1197866.042),15.0,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.2,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.90,52828.501430,4.775581e+07,1.092283,3
1867,27419,1,POINT (688530.507 1197529.777),15.0,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.2,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.90,52828.501430,4.775581e+07,1.092283,3
1868,27545,1,POINT (688245.763 1197879.392),15.0,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.2,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.90,52828.501430,4.775581e+07,1.092283,3


Ref #1: https://www.kite.com/python/answers/how-to-find-the-max-value-in-a-dictionary-in-python

Ref #2: https://samdotson1992.github.io/SuperGIS/blog/k-means-clustering/


#Heatmap

In [ ]:
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

Em tham khảo longitude với latitude của TP.HCM ở trang này ạ

Ref: https://www.latlong.net/place/ho-chi-minh-city-vietnam-333.html

In [ ]:
heatmap = folium.Map(location=[10.762622, 106.660172], zoom_start = 10, tiles='CartoDB positron')
heatmap

In [ ]:
from folium.plugins import HeatMap

In [ ]:
data_filtered.crs

<Projected CRS: EPSG:32648>
Name: WGS 84 / UTM zone 48N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 102°E and 108°E, northern hemisphere between equator and 84°N, onshore and offshore. Cambodia. China. Indonesia. Laos. Malaysia - West Malaysia. Mongolia. Russian Federation. Singapore. Thailand. Vietnam.
- bounds: (102.0, 0.0, 108.0, 84.0)
Coordinate Operation:
- name: UTM zone 48N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
#Chuyển đổi hệ toạ độ của dữ liệu để phù hợp với hệ toạ độ của bản đồ
final_data_use = final_data_use.to_crs(epsg=4326)

In [ ]:
final_data_use.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
lon = pd.Series(final_data_use['geometry'].apply(lambda p: p.x))
lat = pd.Series(final_data_use['geometry'].apply(lambda p: p.y))

In [ ]:
table = []

In [ ]:
#Vì dữ liệu đầu vào của Heatmap là list chứa các list con là [lat, lon] nên thực hiện chuyển đổi dữ liệu 
for each_lat, each_lon in zip(lat, lon):
  temp = [each_lat, each_lon]
  table.append(temp)

In [ ]:
table

[[10.7901163481992, 106.515760343038],
 [10.7908130466793, 106.515019384016],
 [10.790356258077196, 106.513686834416],
 [10.7907578192849, 106.512395537323],
 [10.772755610958, 106.524237152807],
 [10.789781279103998, 106.515623008566],
 [10.7907352626062, 106.515344047901],
 [10.787803685910202, 106.517984569073],
 [10.790585643166597, 106.515324289992],
 [10.7921006357007, 106.513401973938],
 [10.7906727696843, 106.512496220657],
 [10.791460693982602, 106.51261270088901],
 [10.792425580354697, 106.512347645705],
 [10.790428277964699, 106.515731303304],
 [10.792068324138999, 106.513453353408],
 [10.790761830197798, 106.515275777168],
 [10.790534677540796, 106.515460901387],
 [10.792263825507199, 106.513253093799],
 [10.777547330320298, 106.514462802492],
 [10.786794553272001, 106.519746780396],
 [10.7911296909058, 106.512163714052],
 [10.790049560751498, 106.51573273568],
 [10.7919977601542, 106.511582918476],
 [10.7902249159842, 106.513587576028],
 [10.790739924743598, 106.5139328400

In [ ]:
data_filtered = data_filtered.to_crs(epsg=4326)

In [ ]:
for _, r in data_filtered.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['Dist_Name']).add_to(geo_j)
    geo_j.add_to(heatmap)
heatmap

In [ ]:
HeatMap(table).add_to(folium.FeatureGroup(name='Heat Map').add_to(heatmap))
heatmap

Ref: https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data

#Lưu lại thành file PNG

In [ ]:
#Chuyển file về định dạng html trước
heatmap.save('heatmap.html')

In [ ]:
!pip install selenium

     |████████████████████████████████| 904 kB 7.3 MB/s 


In [ ]:
from selenium import webdriver
import time

In [ ]:
!apt-get update 
!apt install chromium-chromedriver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [68.5 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


Ref: https://www.py4u.net/discuss/18715

In [ ]:
driver.set_window_size(4000, 3000) 
driver.get('file:///C:/Users/Mai/Downloads/heatmap.html')

time.sleep(5)
driver.save_screenshot('heatmap.png')
#Vẫn chưa lưu lại được thành file PNG 

True

#Kết luận 

Đã thực hiện được đến bước lưu lại cụm điểm cluster lớn nhất của mỗi quận huyện nhưng vẫn chưa thể tạo heatmap và lưu lại file dưới dạng PNG

Cập nhật lúc: 3:50 PM ngày 30/9/2021

Đã có thể vẽ được heatmap và lưu lại thành định dạng .html, vẫn chưa lưu được thành định dạng .png